# ADM1 Mass Balance Analysis Notebook

This notebook performs detailed mass balance calculations on ADM1 simulation outputs, including COD, elemental (C & N), gas–liquid transfer, volatile solids, and sensitivity to operational parameters.

Sections:
1. Imports & Environment
2. Load / Generate Simulation Data
3. Assemble Time-Series Frames
4. Species Metadata & Factors
5. Core Mass Balance Function
6. Aggregate & Closure Metrics
7. Elemental & COD Balances
8. Gas–Liquid Transfer
9. Solids / VS Tracking
10. Adaptive Time-Step Handling
11. Residual Visualizations
12. Closure Heatmap
13. Anomaly Flagging
14. Sensitivity Sweep
15. Export Results
16. Reusable API
17. Test Snippets
18. CLI Hook
19. Performance Profiling

In [1]:
# 1) Imports & setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from adm1.coAD import ADM1_coAD
from adm1.influent import get_influent
from adm1.initial_state import get_initial_state
from adm1.params import get_VSS

pd.set_option('display.precision', 6)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context('notebook')
print('Imports ready')

Imports ready


In [2]:
# 2) Run a basic ADM1 simulation
q_ad_init = 193.3           # m^3/d (example)
density = 1.0               # tonne/m^3 (assume water-like)
VS_per_TS = 0.99           # kg VS / kg TS (example)
TS_fraction = 0.10          # fraction (-)
mixing_ratio = 0.99999999          # example mix ratio
OLR = 4.0                   # kg VS / m^3 / d (example)
T_ad = 308.15               # K
T_base = 298.15             # K
T_op = T_ad
recycle_ratio = 0.0

# Influent & initials (optional explicit values)
influent = get_influent(mixing_ratio)
initials = get_initial_state(mixing_ratio)

print('Running ADM1_coAD...')
result = ADM1_coAD(
    q_ad_init,                       # Initial influent flow rate [m^3/d]
    density,                         # Influent density [tonne/m^3]
    VS_per_TS,                       # Volatile solids per total solids [kg VS/kg TS]     
    TS_fraction,                     # Fraction of TS in influent
    mixing_ratio,                     # Fraction for feed 1 / total (can be overridden per scenario)
    OLR,                             # Organic Loading Rate [kg VS/m3/d]
    T_ad,                            # K
    T_base,                          # K
    T_op,                            # K
    recycle_ratio,
    influent=None,                   # Optional: pass custom influent dict
    initials=None,                   # Optional: pass custom initial state dict
    VSS=None,                        # Optional: pass custom VSS value
    param_overrides=None,            # Optional: dict of parameter overrides (e.g., k_L_a, k_p, K_H_*)
    disable_inhibition=False,        # run with inhibition disabled (all I_* = 1)     
)

simulate_results = result['simulate_results']
mixed_influent_history = result.get('mixed_influent_history')
print('Done. simulate_results rows:', len(simulate_results))

Running ADM1_coAD...
Done. simulate_results rows: 2400


In [3]:
# 6) Print simulate_results at the last time step
import pandas as pd

if 'simulate_results' not in globals() or simulate_results is None or simulate_results.empty:
    print('simulate_results is empty or not defined')
else:
    last_row = simulate_results.iloc[[-1]]  # keep as DataFrame to show all columns
    try:
        from IPython.display import display
        display(last_row)
    except Exception:
        print(last_row.to_string(index=False))

,S_su,S_aa,S_fa,S_va,S_bu,S_pro,S_ac,S_h2,S_ch4,S_IC,...,S_bu_ion,S_pro_ion,S_ac_ion,S_hco3_ion,S_co2,S_nh3,S_nh4_ion,S_gas_h2,S_gas_ch4,S_gas_co2
2399,0.009593,0.004262,0.077114,0.017386,0.025041,0.012942,0.033627,2.054556e-07,0.054445,0.065188,...,0.024847,0.012827,0.0334,0.052599,0.012589,0.000431,0.045833,0.000009,1.65129,0.017995


In [4]:
# 7) Print mixed_influent (last) using the requested pattern
try:
    res = result
except NameError:
    res = {}

mih = res.get('mixed_influent_history', None)
if mih is not None and hasattr(mih, 'iloc') and len(mih) > 0:
    mixed_last_row = mih.iloc[-1].to_dict()
else:
    mixed_last_row = {}

print('mixed_influent (last):')
print(mixed_last_row)

mixed_influent (last):
{'time': 100.0, 'S_su_in': 0.000370981830510536, 'S_aa_in': 0.000370981830510536, 'S_fa_in': 0.000370981830510536, 'S_va_in': 0.000370981830510536, 'S_bu_in': 0.000370981830510536, 'S_pro_in': 0.000370981830510536, 'S_ac_in': 0.000370981830510536, 'S_h2_in': 3.709818305105359e-09, 'S_ch4_in': 3.7098183051053597e-06, 'S_co2_in': 3.7098183051053597e-06, 'S_IC_in': 0.014839273220421436, 'S_IN_in': 0.0, 'S_I_in': 0.007419636610210718, 'X_xc1_in': 0.0, 'X_ch1_in': 19.73994300406618, 'X_pr1_in': 12.472409017040123, 'X_li1_in': 9.834728228487021, 'X_xc2_in': 0.0, 'X_ch2_in': 7.619732471221038e-07, 'X_pr2_in': 1.7747224996261654e-07, 'X_li2_in': 2.5077600538195817e-08, 'X_su_in': 0.0, 'X_aa_in': 0.0, 'X_fa_in': 0.0, 'X_c4_in': 0.0, 'X_pro_in': 0.0, 'X_ac_in': 0.0, 'X_h2_in': 0.0, 'X_I_in': 83.41526382671323, 'S_cation_in': 0.014839273220421436, 'S_anion_in': 0.007419636610210718, 'S_nh3_in': 0.0}


In [5]:
# 8) Side-by-side table: mixed_influent (last) vs simulate_results (last)
import pandas as pd
import numpy as np

# Prepare simulate_results last row (keep as dict, but drop 'time' if present)
if 'simulate_results' not in globals() or simulate_results is None or simulate_results.empty:
    eff_last_row = {}
else:
    _df = simulate_results
    eff_last_row = _df.iloc[-1].to_dict()
    eff_last_row.pop('time', None)

# Prepare mixed influent last row using the requested pattern
try:
    res = result
except NameError:
    res = {}

mih = res.get('mixed_influent_history', None)
if mih is not None and hasattr(mih, 'iloc') and len(mih) > 0:
    mixed_last_row = mih.iloc[-1].to_dict()
else:
    mixed_last_row = {}

# Normalize influent keys by stripping trailing '_in'
def base_name(k: str):
    return k[:-3] if isinstance(k, str) and k.endswith('_in') else k

# Determine all species to include
all_keys = set()
for d in (mixed_last_row, eff_last_row):
    for k in getattr(d, 'keys', lambda: [])():
        all_keys.add(base_name(k))

# Build rows
rows = []
for bk in sorted(all_keys):
    def pick(d, suffix_ok=True):
        if not d:
            return np.nan
        if bk in d:
            return d[bk]
        if suffix_ok:
            ki = f"{bk}_in"
            if ki in d:
                return d[ki]
        return np.nan

    v_mix = pick(mixed_last_row)
    v_eff = pick(eff_last_row, suffix_ok=False)

    def to_float(x):
        try:
            if isinstance(x, (list, tuple, np.ndarray)):
                if len(x) == 0:
                    return np.nan
                x = x[-1]
            return float(x)
        except Exception:
            return np.nan

    rows.append({
        'species': bk,
        'mixed_influent': to_float(v_mix),
        'simulate_results': to_float(v_eff),
    })

comp_table = pd.DataFrame(rows)

# Drop rows with both NaN
mask_all_nan = comp_table[['mixed_influent','simulate_results']].isna().all(axis=1)
comp_table = comp_table[~mask_all_nan].reset_index(drop=True)

# Exclusions
ion_exclusions = {
    'S_ac_ion','S_bu_ion','S_va_ion','S_pro_ion','S_hco3_ion','S_nh3','S_nh4_ion',
    'S_cation','S_anion','S_co2'
}
comp_table = comp_table[~comp_table['species'].isin(ion_exclusions)].reset_index(drop=True)
comp_table = comp_table[~comp_table['species'].isin({'pH','time'})].reset_index(drop=True)

# Ordering
priority_order = [
    'S_su','S_aa','S_fa','S_va','S_bu','S_pro','S_ac','S_h2','S_ch4','S_IC','S_IN','S_I',
    'X_su','X_aa','X_fa','X_c4','X_pro','X_ac','X_h2','X_I'
]
gas_species = {'S_gas_ch4','S_gas_co2','S_gas_h2'}

non_gas = comp_table[~comp_table['species'].isin(gas_species)].copy()
remaining = [k for k in non_gas['species'] if k not in priority_order]
ordered_non_gas = priority_order + remaining
non_gas['species_rank'] = non_gas['species'].apply(lambda x: ordered_non_gas.index(x) if x in ordered_non_gas else 10**6)
non_gas = non_gas.sort_values('species_rank').drop(columns='species_rank')

gas_rows = comp_table[comp_table['species'].isin(gas_species)].copy()
if not gas_rows.empty:
    gas_order = ['S_gas_ch4','S_gas_co2','S_gas_h2']
    gas_rows['gas_rank'] = gas_rows['species'].apply(lambda x: gas_order.index(x) if x in gas_order else 10**3)
    gas_rows = gas_rows.sort_values('gas_rank').drop(columns='gas_rank')

comp_table = pd.concat([non_gas, gas_rows], ignore_index=True)

# Unit mapping
unit_map = {
    'S_su':'kgCOD/m3','S_aa':'kgCOD/m3','S_fa':'kgCOD/m3','S_va':'kgCOD/m3','S_bu':'kgCOD/m3',
    'S_pro':'kgCOD/m3','S_ac':'kgCOD/m3','S_h2':'kgCOD/m3','S_ch4':'kgCOD/m3',
    'S_IC':'kmol/m3','S_IN':'kmol/m3','S_I':'kgCOD/m3',
    'X_su':'kgCOD/m3','X_aa':'kgCOD/m3','X_fa':'kgCOD/m3','X_c4':'kgCOD/m3','X_pro':'kgCOD/m3',
    'X_ac':'kgCOD/m3','X_h2':'kgCOD/m3','X_I':'kgCOD/m3',
    'X_xc1':'kgCOD/m3','X_ch1':'kgCOD/m3','X_pr1':'kgCOD/m3','X_li1':'kgCOD/m3',
    'X_xc2':'kgCOD/m3','X_ch2':'kgCOD/m3','X_pr2':'kgCOD/m3','X_li2':'kgCOD/m3',
    'S_gas_ch4':'kgCOD/m3','S_gas_co2':'kmol/m3','S_gas_h2':'kgCOD/m3'
}
comp_table['unit'] = comp_table['species'].map(unit_map).fillna('')

# Reorder and format
comp_table = comp_table[['species','unit','mixed_influent','simulate_results']]

formatted = comp_table.copy()

def _fmt(x):
    try:
        if pd.isna(x):
            return ''
        return f"{float(x):.3f}"
    except Exception:
        return ''

for c in ['mixed_influent','simulate_results']:
    formatted[c] = formatted[c].apply(_fmt)

try:
    from IPython.display import display
    display(formatted)
except Exception:
    print(formatted.to_string(index=False))

print(f"Rows in side-by-side table: {len(comp_table)}")

,species,unit,mixed_influent,simulate_results
0,S_su,kgCOD/m3,0.000,0.010
1,S_aa,kgCOD/m3,0.000,0.004
2,S_fa,kgCOD/m3,0.000,0.077
3,S_va,kgCOD/m3,0.000,0.017
4,S_bu,kgCOD/m3,0.000,0.025
5,S_pro,kgCOD/m3,0.000,0.013
6,S_ac,kgCOD/m3,0.000,0.034
7,S_h2,kgCOD/m3,0.000,0.000
8,S_ch4,kgCOD/m3,0.000,0.054
9,S_IC,kmol/m3,0.015,0.065


Rows in side-by-side table: 31


In [6]:
# 9) Overall mass balance (rates and closure)
import pandas as pd
import numpy as np

# Ensure we have the side-by-side table; otherwise rebuild a minimal version
if 'comp_table' not in globals() or comp_table is None or comp_table.empty:
    # Minimal rebuild using the logic from cell 8
    # Prepare simulate_results last row (drop 'time' if present)
    if 'simulate_results' not in globals() or simulate_results is None or simulate_results.empty:
        eff_last_row = {}
    else:
        _df = simulate_results
        eff_last_row = _df.iloc[-1].to_dict()
        eff_last_row.pop('time', None)

    # mixed influent via result
    try:
        res = result
    except NameError:
        res = {}
    mih = res.get('mixed_influent_history', None)
    if mih is not None and hasattr(mih, 'iloc') and len(mih) > 0:
        mixed_last_row = mih.iloc[-1].to_dict()
    else:
        mixed_last_row = {}

    def base_name(k: str):
        return k[:-3] if isinstance(k, str) and k.endswith('_in') else k

    all_keys = set()
    for d in (mixed_last_row, eff_last_row):
        for k in getattr(d, 'keys', lambda: [])():
            all_keys.add(base_name(k))

    rows = []
    for bk in sorted(all_keys):
        def pick(d, suffix_ok=True):
            if not d:
                return np.nan
            if bk in d:
                return d[bk]
            if suffix_ok:
                ki = f"{bk}_in"
                if ki in d:
                    return d[ki]
            return np.nan
        def to_float(x):
            try:
                if isinstance(x, (list, tuple, np.ndarray)):
                    if len(x) == 0:
                        return np.nan
                    x = x[-1]
                return float(x)
            except Exception:
                return np.nan
        rows.append({
            'species': bk,
            'mixed_influent': to_float(pick(mixed_last_row)),
            'simulate_results': to_float(pick(eff_last_row, suffix_ok=False)),
        })

    comp_table = pd.DataFrame(rows)

    ion_exclusions = {
        'S_ac_ion','S_bu_ion','S_va_ion','S_pro_ion','S_hco3_ion','S_nh3','S_nh4_ion',
        'S_cation','S_anion','S_co2'
    }
    comp_table = comp_table[~comp_table['species'].isin(ion_exclusions)]
    comp_table = comp_table[~comp_table['species'].isin({'pH','time'})].reset_index(drop=True)

    unit_map = {
        'S_su':'kgCOD/m3','S_aa':'kgCOD/m3','S_fa':'kgCOD/m3','S_va':'kgCOD/m3','S_bu':'kgCOD/m3',
        'S_pro':'kgCOD/m3','S_ac':'kgCOD/m3','S_h2':'kgCOD/m3','S_ch4':'kgCOD/m3',
        'S_IC':'kmol/m3','S_IN':'kmol/m3','S_I':'kgCOD/m3',
        'X_su':'kgCOD/m3','X_aa':'kgCOD/m3','X_fa':'kgCOD/m3','X_c4':'kgCOD/m3','X_pro':'kgCOD/m3',
        'X_ac':'kgCOD/m3','X_h2':'kgCOD/m3','X_I':'kgCOD/m3',
        'X_xc1':'kgCOD/m3','X_ch1':'kgCOD/m3','X_pr1':'kgCOD/m3','X_li1':'kgCOD/m3',
        'X_xc2':'kgCOD/m3','X_ch2':'kgCOD/m3','X_pr2':'kgCOD/m3','X_li2':'kgCOD/m3',
        'S_gas_ch4':'kgCOD/m3','S_gas_co2':'kmol/m3','S_gas_h2':'kgCOD/m3'
    }
    comp_table['unit'] = comp_table['species'].map(unit_map).fillna('')

# Determine flows (m3/d)
try:
    res = result
except NameError:
    res = {}

q_in_val = res.get('q_in', np.nan)
q_ad_val = res.get('q_ad', q_in_val)
q_eff_val = q_ad_val  # assume equal to feed at last step (no accumulation considered here)

# Fallbacks from notebook variables
if not np.isfinite(q_ad_val):
    q_ad_val = 'q_ad_init' in globals() and float(q_ad_init) or np.nan
if not np.isfinite(q_eff_val):
    q_eff_val = q_ad_val

# Compute rates per species
mb = comp_table.copy()
mb['rate_unit'] = ''
mb['In_rate'] = np.nan
mb['Out_rate'] = np.nan

kg_mask = mb['unit'].eq('kgCOD/m3')
kmol_mask = mb['unit'].eq('kmol/m3')

# In_rate = mixed_influent * q_ad; Out_rate = simulate_results * q_eff
mb.loc[kg_mask, 'rate_unit'] = 'kgCOD/d'
mb.loc[kg_mask, 'In_rate'] = mb.loc[kg_mask, 'mixed_influent'] * float(q_ad_val) if np.isfinite(q_ad_val) else np.nan
mb.loc[kg_mask, 'Out_rate'] = mb.loc[kg_mask, 'simulate_results'] * float(q_eff_val) if np.isfinite(q_eff_val) else np.nan

mb.loc[kmol_mask, 'rate_unit'] = 'kmol/d'
mb.loc[kmol_mask, 'In_rate'] = mb.loc[kmol_mask, 'mixed_influent'] * float(q_ad_val) if np.isfinite(q_ad_val) else np.nan
mb.loc[kmol_mask, 'Out_rate'] = mb.loc[kmol_mask, 'simulate_results'] * float(q_eff_val) if np.isfinite(q_eff_val) else np.nan

# Residuals and closure metrics
mb['Residual'] = mb['In_rate'] - mb['Out_rate']
_eps = 1e-12
mb['Residual_%_of_In'] = np.where(np.abs(mb['In_rate']) > _eps, (mb['Residual'] / mb['In_rate']) * 100.0, np.nan)

# Order columns
mb = mb[['species','unit','rate_unit','mixed_influent','simulate_results','In_rate','Out_rate','Residual','Residual_%_of_In']]

# Totals by rate unit
totals = mb.groupby('rate_unit', dropna=False)[['In_rate','Out_rate','Residual']].sum(min_count=1).reset_index()

# Formatting for display
mb_fmt = mb.copy()
for c in ['mixed_influent','simulate_results','In_rate','Out_rate','Residual','Residual_%_of_In']:
    mb_fmt[c] = mb_fmt[c].apply(lambda x: '' if pd.isna(x) else f"{x:.3f}")

totals_fmt = totals.copy()
for c in ['In_rate','Out_rate','Residual']:
    totals_fmt[c] = totals_fmt[c].apply(lambda x: '' if pd.isna(x) else f"{x:.3f}")

try:
    from IPython.display import display
    print('Overall mass balance per species:')
    display(mb_fmt)
    print('Totals by unit:')
    display(totals_fmt)
except Exception:
    print('Overall mass balance per species:')
    print(mb_fmt.to_string(index=False))
    print('\nTotals by unit:')
    print(totals_fmt.to_string(index=False))

Overall mass balance per species:


,species,unit,rate_unit,mixed_influent,simulate_results,In_rate,Out_rate,Residual,Residual_%_of_In
0,S_su,kgCOD/m3,kgCOD/d,0.000,0.010,0.193,4.998,-4.805,-2485.787
1,S_aa,kgCOD/m3,kgCOD/d,0.000,0.004,0.193,2.221,-2.027,-1048.803
2,S_fa,kgCOD/m3,kgCOD/d,0.000,0.077,0.193,40.180,-39.987,-20686.562
3,S_va,kgCOD/m3,kgCOD/d,0.000,0.017,0.193,9.059,-8.866,-4586.410
4,S_bu,kgCOD/m3,kgCOD/d,0.000,0.025,0.193,13.048,-12.854,-6649.873
5,S_pro,kgCOD/m3,kgCOD/d,0.000,0.013,0.193,6.744,-6.550,-3388.686
6,S_ac,kgCOD/m3,kgCOD/d,0.000,0.034,0.193,17.522,-17.328,-8964.416
7,S_h2,kgCOD/m3,kgCOD/d,0.000,0.000,0.000,0.000,-0.000,-5438.157
8,S_ch4,kgCOD/m3,kgCOD/d,0.000,0.054,0.002,28.369,-28.367,-1467498.487
9,S_IC,kmol/m3,kmol/d,0.015,0.065,7.732,33.966,-26.234,-339.294


Totals by unit:


,rate_unit,In_rate,Out_rate,Residual
0,kgCOD/d,65377.348,52535.091,13702.666
1,kmol/d,7.732,67.448,-50.340
